# Testing MIRISim PA and RA Dec


In [2]:
import os, sys, re, shutil, glob, time, datetime, json, yaml, asdf
os.environ["CRDS_PATH"] = os.path.expanduser('~/Data/JWST-CRDS') # '/n17data/dzl
os.environ["CRDS_SERVER_URL"] = 'https://jwst-crds.stsci.edu'
os.environ["MIRAGE_DATA"] = os.path.expanduser('~/Data/JWST-MIRAGE') # '/n23da
os.environ["MIRISIM_ROOT"] = os.path.expanduser('~/Data/JWST-MIRISIM/mirisim')
os.environ["PYSYN_CDBS"] = os.path.expanduser('~/Data/JWST-MIRISIM/mirisim/cdbs/')
os.environ["CDP_DIR"] = os.path.expanduser('~/Data/JWST-MIRISIM/CDP')
#sys.path.insert(0, os.path.expanduser('~/Cloud/Github/mirage'))
#sys.path.insert(0, os.path.expanduser('~/Cloud/Github/mirisim_iap_fr'))
import astropy.units as u
import numpy as np
import logging
logging.basicConfig(level='DEBUG')
import pysiaf

import configobj # needed by mirisim
import pysynphot # needed by mirisim
import importlib_resources # needed by mirisim
import pysftp # needed by mirisim
import pySpecSim # needed by mirisim, copied from server, cannot be installed via pip

import mirisim
from mirisim import MiriSimulation
from mirisim.config_parser import SimConfig, SceneConfig, SimulatorConfig
from mirisim.imsim.ima import get_ima_subarray_to_skysim_transform
from mirisim.imsim.mirimImager import MirimImager
from mirisim.obssim import ObservationSimulation
from mirisim.obssim.pointing import Pointing
from mirisim.skysim import builder
from mirisim.skysim import scenemaker
from mirisim.skysim.catalogues import Catalogue
from mirisim.skysim.scenes import SkyScene, CompositeSkyScene
from mirisim.skysim.astrosources import Galaxy, Star
from mirisim.skysim.disks import SersicDisk
from mirisim.skysim.externalsources import Skycube, Skyimage


In [3]:
# APT pointing file, exported from APT

apt_pointing_table_text = '''\
Tar Tile Exp Dith Aperture Name             Target       RA         Dec        BaseX    BaseY    DithX    DithY    V2       V3       IdlX     IdlY   Level      Type      ExPar  DkPar   dDist
  1    1   1    1 NRCALL_FULL               19 CWEBTIL +149.92961   +2.43173   +0.000   +0.000  -24.600   -3.100  +22.442 -495.969  -24.600   -3.100 TARGET     SCIENCE        0      0  0.000 (base)
  1    1   1    1 MIRIM_ILLUM               19 CWEBTIL +149.92961   +2.43173 -459.648  -80.643  -24.776   -0.973  +22.442 -495.969 -484.424  -81.615 TARGET     PARALLEL       0      0
  1    1   1    2 NRCALL_FULL               19 CWEBTIL +149.92961   +2.43173   +0.000   +0.000  -24.800   +2.900  +22.632 -489.969  -24.800   +2.900 DITHER     SCIENCE        0      0  6.003
  1    1   1    2 MIRIM_ILLUM               19 CWEBTIL +149.92961   +2.43173 -459.648  -80.643  -24.459   +5.022  +22.632 -489.969 -484.108  -75.620 DITHER     PARALLEL       0      0
  1    1   1    3 NRCALL_FULL               19 CWEBTIL +149.92961   +2.43173   +0.000   +0.000  +24.600   +3.100  -26.769 -489.853  +24.600   +3.100 DITHER     SCIENCE        0      0 49.400
  1    1   1    3 MIRIM_ILLUM               19 CWEBTIL +149.92961   +2.43173 -459.648  -80.643  +24.776   +0.973  -26.769 -489.853 -434.873  -79.669 DITHER     PARALLEL       0      0
  1    1   1    4 NRCALL_FULL               19 CWEBTIL +149.92961   +2.43173   +0.000   +0.000  +24.800   -2.900  -26.959 -495.853  +24.800   -2.900 DITHER     SCIENCE        0      0  6.003
  1    1   1    4 MIRIM_ILLUM               19 CWEBTIL +149.92961   +2.43173 -459.648  -80.643  +24.459   -5.022  -26.959 -495.853 -435.190  -85.664 DITHER     PARALLEL       0      0
  1    1   2    1 NRCALL_FULL               19 CWEBTIL +149.92961   +2.43173   +0.000   +0.000  -24.600   -3.100  +22.442 -495.969  -24.600   -3.100 FILTER     SCIENCE        0      0 49.400
  1    1   2    1 MIRIM_ILLUM               19 CWEBTIL +149.92961   +2.43173 -459.648  -80.643  -24.776   -0.973  +22.442 -495.969 -484.424  -81.615 FILTER     PARALLEL       0      0
  1    1   2    2 NRCALL_FULL               19 CWEBTIL +149.92961   +2.43173   +0.000   +0.000  -24.800   +2.900  +22.632 -489.969  -24.800   +2.900 DITHER     SCIENCE        0      0  6.003
  1    1   2    2 MIRIM_ILLUM               19 CWEBTIL +149.92961   +2.43173 -459.648  -80.643  -24.459   +5.022  +22.632 -489.969 -484.108  -75.620 DITHER     PARALLEL       0      0
  1    1   2    3 NRCALL_FULL               19 CWEBTIL +149.92961   +2.43173   +0.000   +0.000  +24.600   +3.100  -26.769 -489.853  +24.600   +3.100 DITHER     SCIENCE        0      0 49.400
  1    1   2    3 MIRIM_ILLUM               19 CWEBTIL +149.92961   +2.43173 -459.648  -80.643  +24.776   +0.973  -26.769 -489.853 -434.873  -79.669 DITHER     PARALLEL       0      0
  1    1   2    4 NRCALL_FULL               19 CWEBTIL +149.92961   +2.43173   +0.000   +0.000  +24.800   -2.900  -26.959 -495.853  +24.800   -2.900 DITHER     SCIENCE        0      0  6.003
'''

# Notes: 'DithX' 'DithY' are the same as the 'xoffset' 'yoffset' in MIRAGE YAML files.
# They are Ideal Coordinates, see `dither.x_offset`, `xv, yv = idl2v23(xoffset, yoffset)` ("wavecorr.py"). 

# Read this APT Pointing table
from collections import OrderedDict
apt_pointing_table_header = None
apt_pointing_table_data = OrderedDict()
for line in apt_pointing_table_text.split('\n'):
    line = line.strip()
    if line == '':
        continue
    elif line.startswith('#'):
        continue
    if apt_pointing_table_header is None:
        line = line.replace('Aperture Name', 'Aperture_Name')
        line = line.replace('Target', 'ID Target')
        line += ' base'
        apt_pointing_table_header = line.split()
        for key in apt_pointing_table_header:
            apt_pointing_table_data[key] = []
    else:
        items = line.split()
        for i, key in enumerate(apt_pointing_table_header):
            if i < len(items):
                apt_pointing_table_data[key].append(items[i])
            else:
                apt_pointing_table_data[key].append('')

# Deal with data types
for key in apt_pointing_table_header:
    try: 
        flt_arr = np.array(apt_pointing_table_data[key]).astype(float)
        int_arr = np.floor(flt_arr)
        if np.all(np.isclose(flt_arr, int_arr, rtol=1e-5, atol=1e-10)):
            apt_pointing_table_data[key] = int_arr.astype(int)
        else:
            apt_pointing_table_data[key] = flt_arr
    except:
        pass

# Make it an astropy Table
from astropy.table import Table
apt_pointing_table = Table(apt_pointing_table_data)
apt_pointing_table

Tar,Tile,Exp,Dith,Aperture_Name,ID,Target,RA,Dec,BaseX,BaseY,DithX,DithY,V2,V3,IdlX,IdlY,Level,Type,ExPar,DkPar,dDist,base
int64,int64,int64,int64,str11,int64,str7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str6,str8,int64,int64,str6,str6
1,1,1,1,NRCALL_FULL,19,CWEBTIL,149.92961,2.43173,0.0,0.0,-24.6,-3.1,22.442,-495.969,-24.6,-3.1,TARGET,SCIENCE,0,0,0.000,(base)
1,1,1,1,MIRIM_ILLUM,19,CWEBTIL,149.92961,2.43173,-459.648,-80.643,-24.776,-0.973,22.442,-495.969,-484.424,-81.615,TARGET,PARALLEL,0,0,,
1,1,1,2,NRCALL_FULL,19,CWEBTIL,149.92961,2.43173,0.0,0.0,-24.8,2.9,22.632,-489.969,-24.8,2.9,DITHER,SCIENCE,0,0,6.003,
1,1,1,2,MIRIM_ILLUM,19,CWEBTIL,149.92961,2.43173,-459.648,-80.643,-24.459,5.022,22.632,-489.969,-484.108,-75.62,DITHER,PARALLEL,0,0,,
1,1,1,3,NRCALL_FULL,19,CWEBTIL,149.92961,2.43173,0.0,0.0,24.6,3.1,-26.769,-489.853,24.6,3.1,DITHER,SCIENCE,0,0,49.400,
1,1,1,3,MIRIM_ILLUM,19,CWEBTIL,149.92961,2.43173,-459.648,-80.643,24.776,0.973,-26.769,-489.853,-434.873,-79.669,DITHER,PARALLEL,0,0,,
1,1,1,4,NRCALL_FULL,19,CWEBTIL,149.92961,2.43173,0.0,0.0,24.8,-2.9,-26.959,-495.853,24.8,-2.9,DITHER,SCIENCE,0,0,6.003,
1,1,1,4,MIRIM_ILLUM,19,CWEBTIL,149.92961,2.43173,-459.648,-80.643,24.459,-5.022,-26.959,-495.853,-435.19,-85.664,DITHER,PARALLEL,0,0,,
1,1,2,1,NRCALL_FULL,19,CWEBTIL,149.92961,2.43173,0.0,0.0,-24.6,-3.1,22.442,-495.969,-24.6,-3.1,FILTER,SCIENCE,0,0,49.400,


In [4]:
# Set telescope roll PA V3

telescope_v3_pa = 293.09730273
telescope_v1_ra = apt_pointing_table['RA'][0] # all RA must be the same in a dither sequence
telescope_v1_dec = apt_pointing_table['Dec'][0] # all Dec must be the same in a dither sequence


In [5]:
# Get dither sequence

dither_IdealXY_arcsec = []
for i in range(len(apt_pointing_table)):
    if apt_pointing_table['Aperture_Name'][i]=='MIRIM_ILLUM':
        dither_IdealXY_arcsec.append([apt_pointing_table['DithX'][i], apt_pointing_table['DithY'][i]])

# DEBUG: prepending 0 to check the starting point
dither_IdealXY_arcsec.insert(0, [0., 0.])

# Make it a numpy array
dither_IdealXY_arcsec = np.array(dither_IdealXY_arcsec)
print('dither_IdealXY_arcsec:\n', dither_IdealXY_arcsec)

# Prepare to convert dithers from Ideal Coordinate (idlx,idxy) [arcsec] 
#   to Telescope Coordinate (v2,v3) [arcsec] 
#   then to Science Coordinate / Pixel Focal Plane SCA

import pysiaf
from mirage.utils import siaf_interface

# create siaf object
MIRI_SIAF = pysiaf.siaf.Siaf('MIRI')
MIRIM_FULL_SIAF = MIRI_SIAF.apertures['MIRIM_FULL']

# following `mirage.apt.apt_inputs()` to use `siaf_interface` to get `attitude_matrix`
local_roll, attitude_matrix, fullframesize, subarray_boundaries = \
    siaf_interface.get_siaf_information(
        MIRI_SIAF, 'MIRIM_FULL', telescope_v1_ra, telescope_v1_dec, telescope_v3_pa,
        v2_arcsec = MIRIM_FULL_SIAF.V2Ref, 
        v3_arcsec = MIRIM_FULL_SIAF.V3Ref, # v2 v3 ref of MIRI FULL center
    )
print('local_roll:', local_roll) # V3 PA at local MIRI center location, tiny difference from telescope V3 PA
print('MIRIM_FULL_SIAF.V2Ref, MIRIM_FULL_SIAF.V3Ref:', MIRIM_FULL_SIAF.V2Ref, MIRIM_FULL_SIAF.V3Ref)

# set attitude_matrix so that we can use SIAF.sky_to_tel()
MIRIM_FULL_SIAF.set_attitude_matrix(attitude_matrix)

# check MIRI ILLUM center Ideal coordinate (idl) and the corresponding Pixel coordinate (sci) using pysiaf
print('idl_to_sci({:.3f}, {:.3f}) -> {:.3f} {:.3f}'
      .format(0.0, 0.0, *MIRIM_FULL_SIAF.idl_to_sci(0.0, 0.0))
     )

# another way to check MIRI ILLUM center pixel coordinate is to use mirisim function
from mirisim.imsim.ima import get_ima_colrow_ref
col_ref, row_ref = get_ima_colrow_ref('FULL')
print('col_ref, row_ref: {:.3f}, {:.3f} -- MIRI ILLUM center, in Pixel Focal Plane SCA pixels'
      .format(col_ref, row_ref))
scix_ref, sciy_ref = col_ref+1, row_ref+1 # there is a one-pixel difference, I guess it's because 0-/1- indexing
print('scix_ref, sciy_ref:', scix_ref, sciy_ref)
v2_ref, v3_ref = MIRIM_FULL_SIAF.sci_to_tel(scix_ref, sciy_ref)
print('v2_ref, v3_ref:', v2_ref, v3_ref)
idlx_ref, idly_ref = MIRIM_FULL_SIAF.sci_to_idl(scix_ref, sciy_ref)
print('idlx_ref, idly_ref:', idlx_ref, idly_ref)

# convert dither values from Ideal Coordinate to v2 v3 coordinate and sci pixel coordinate
# mirisim uses the sci pixel coordinate in "dither.dat" file!
ditherXY_arcsec = [] # v2,v3
ditherXY_pixels = [] # sci pix
for i in range(dither_IdealXY_arcsec.shape[0]): 
    # 
    idlx_dither, idly_dither = dither_IdealXY_arcsec[i, 0], dither_IdealXY_arcsec[i, 1]
    v2_dither, v3_dither = MIRIM_FULL_SIAF.idl_to_tel(
        idlx_dither + idlx_ref, 
        idly_dither + idly_ref
    )
    v2_dither, v3_dither = v2_dither - v2_ref, v3_dither - v3_ref
    print('idl_to_tel({:.3f}, {:.3f}) -> {:.3f} {:.3f} (idl x, y -> tel v2, v3) (mirisim: -v2_off, -v3_off)'
          .format(idlx_dither, idly_dither, 
                  v2_dither, v3_dither)
         )
    # 
    scix_dither, sciy_dither = MIRIM_FULL_SIAF.idl_to_sci(
        idlx_dither + idlx_ref, 
        idly_dither + idly_ref
    )
    scix_dither, sciy_dither = scix_dither - scix_ref, sciy_dither - sciy_ref
    print('idl_to_sci({:.3f}, {:.3f}) -> {:.3f} {:.3f} (idl x, y -> sci x, y)'
          .format(idlx_dither, idly_dither, 
                  scix_dither, sciy_dither)
         )
    ditherXY_arcsec.append([v2_dither, v3_dither])
    ditherXY_pixels.append([scix_dither, sciy_dither])
ditherXY_arcsec = np.array(ditherXY_arcsec)
ditherXY_pixels = np.array(ditherXY_pixels)
print('ditherXY_arcsec:\n', ditherXY_arcsec)
print('ditherXY_pixels:\n', ditherXY_pixels)

dither_IdealXY_arcsec:
 [[  0.      0.   ]
 [-24.776  -0.973]
 [-24.459   5.022]
 [ 24.776   0.973]
 [ 24.459  -5.022]
 [-24.776  -0.973]
 [-24.459   5.022]
 [ 24.776   0.973]]
local_roll: 293.0990826383529
MIRIM_FULL_SIAF.V2Ref, MIRIM_FULL_SIAF.V3Ref: -453.559116 -373.814447
idl_to_sci(0.000, 0.000) -> 693.500 512.500
col_ref, row_ref: 692.500, 511.500 -- MIRI ILLUM center, in Pixel Focal Plane SCA pixels
scix_ref, sciy_ref: 693.5 512.5
v2_ref, v3_ref: -453.559116 -373.814447
idlx_ref, idly_ref: 0.0 0.0
idl_to_tel(0.000, 0.000) -> 0.000 0.000 (idl x, y -> tel v2, v3) (mirisim: -v2_off, -v3_off)
idl_to_sci(0.000, 0.000) -> 0.000 0.000 (idl x, y -> sci x, y)
idl_to_tel(-24.776, -0.973) -> 24.606 -3.058 (idl x, y -> tel v2, v3) (mirisim: -v2_off, -v3_off)
idl_to_sci(-24.776, -0.973) -> -224.442 -9.506 (idl x, y -> sci x, y)
idl_to_tel(-24.459, 5.022) -> 24.795 2.943 (idl x, y -> tel v2, v3) (mirisim: -v2_off, -v3_off)
idl_to_sci(-24.459, 5.022) -> -221.608 44.381 (idl x, y -> sci x, y)
i

In [6]:
# TODO: define functions to add sources into scene.ini

def prepare_sim_file(sim_file):
    # copy from template directory to here
    pass

def prepare_scene_file(scene_file):
    # copy from template directory to here
    pass

def prepare_simulator_file(simulator_file):
    # copy from template directory to here
    pass


In [8]:
# setup mirisim config objects

print('pwd:', os.getcwd())
sim_input_dir = 'test_mirisim_PA_input'
sim_output_dir = 'test_mirisim_PA_output'
sim_file = os.path.join(sim_input_dir, 'ima_simulation.ini')
scene_file = os.path.join(sim_input_dir, 'scene.ini')
simulator_file = os.path.join(sim_input_dir, 'simulator.ini')
dither_file = os.path.join(sim_input_dir, 'dither.dat')
prepare_sim_file(sim_file)
prepare_scene_file(scene_file)
prepare_simulator_file(simulator_file)
sim_config = SimConfig(sim_file)
scene_config = SceneConfig(scene_file)
simulator_config = SimulatorConfig(simulator_file)


pwd: /Users/dzliu/Cloud/Github/Crab.Toolkit.JWST/notebooks


In [9]:
# get pixel scale
# following "imsim/ima.py" `get_ima_v2v3_ref()`

from mirisim.imsim.ima import get_ima_colrow_ref, get_full_to_v2v3_transforms
subarray = sim_config['Integration_and_patterns']['IMA_configuration']['ReadDetect'] # 'FULL'
col_ref, row_ref = get_ima_colrow_ref(subarray)
print('col_ref, row_ref: {:.3f}, {:.3f} -- MIRI FULL center, in Pixel Focal Plane SCA (row, col) in pixels'
      .format(col_ref, row_ref))

filterName = sim_config['Integration_and_patterns']['IMA_configuration']['filter'] # 'F770W'
local_transforms = get_full_to_v2v3_transforms(filterName, simulator_config=simulator_config)
v2_ref, v3_ref = local_transforms.forward_transform(col_ref, row_ref)
print('v2_ref, v3_ref: {:.3f}, {:.3f} -- MIRI FULL center, in JWST Sky Focal Plane (v2, v3) in arcsecond'
      .format(v2_ref, v3_ref))

v2_add, v3_add = local_transforms.forward_transform(col_ref+10., row_ref) # add 10 pixels in X
print('v2_add: {:.3f}, v3_add: {:.3f}'.format(v2_add, v3_add))
v2_pixsc = np.sqrt((v2_add-v2_ref)**2 + (v3_add-v3_ref)**2)/10.
print('v2_pixsc: {:.3f} arcsec'.format(v2_pixsc))

v2_add, v3_add = local_transforms.forward_transform(col_ref, row_ref+10.) # add 10 pixels in Y
print('v2_add: {:.3f}, v3_add: {:.3f}'.format(v2_add, v3_add))
v3_pixsc = np.sqrt((v2_add-v2_ref)**2 + (v3_add-v3_ref)**2)/10.
print('v3_pixsc: {:.3f} arcsec'.format(v3_pixsc))


# get MIRI FULL center RA Dec from telescope pointing RA Dec
# see "obssim/exposure.py" `update_metadata_illum_model()`
ra_ref = telescope_v1_ra + v2_ref/3600.0/np.cos(np.deg2rad(telescope_v1_dec))
dec_ref = telescope_v1_dec + v3_ref/3600.0
print('telescope_v1_ra: {:.7f}, telescope_v1_dec: {:.7f}, ra_ref: {:.7f}, dec_ref: {:.7f}'.format(
    telescope_v1_ra, telescope_v1_dec, ra_ref, dec_ref,
))


col_ref, row_ref: 692.500, 511.500 -- MIRI FULL center, in Pixel Focal Plane SCA (row, col) in pixels


2022-12-08 18:27:48,423 - INFO - Reading 'DISTORTION' model from '/Users/dzliu/Data/JWST-MIRISIM/CDP/MIRI_FM_MIRIMAGE_DISTORTION_07.04.01.fits'


v2_ref, v3_ref: -453.559, -373.814 -- MIRI FULL center, in JWST Sky Focal Plane (v2, v3) in arcsecond
v2_add: -454.660, v3_add: -373.727
v2_pixsc: 0.110 arcsec
v2_add: -453.462, v3_add: -372.706
v3_pixsc: 0.111 arcsec
telescope_v1_ra: 149.9296100, telescope_v1_dec: 2.4317300, ra_ref: 149.8035078, dec_ref: 2.3278927


In [12]:
# prepare mirisim dither file, in pixels

print('Writing to "{}"'.format(dither_file))
with open(dither_file, 'w') as fp:
    for i in range(ditherXY_arcsec.shape[0]):
        # 
        ditherX, ditherY = ditherXY_pixels[i, :]
        print(ditherXY_arcsec[i], '->', ditherX, ditherY)
        # 
        line = '{:+.4f}, {:+.4f}'.format(ditherX, ditherY)
        print(line)
        fp.write(line+'\n')


Writing to "test_mirisim_PA_input/dither.dat"
[0. 0.] -> 0.0 0.0
+0.0000, +0.0000
[24.60586507 -3.05750051] -> -224.44193686795973 -9.506024145349386
-224.4419, -9.5060
[24.79521276  2.94288793] -> -221.60809088104725 44.38067243860746
-221.6081, +44.3807
[-24.60586507   3.05750051] -> 224.37784914260965 10.208040113919878
+224.3778, +10.2080
[-24.79521276  -2.94288793] -> 221.20149439291788 -43.78851226374633
+221.2015, -43.7885
[24.60586507 -3.05750051] -> -224.44193686795973 -9.506024145349386
-224.4419, -9.5060
[24.79521276  2.94288793] -> -221.60809088104725 44.38067243860746
-221.6081, +44.3807
[-24.60586507   3.05750051] -> 224.37784914260965 10.208040113919878
+224.3778, +10.2080


In [13]:
# set dither file path in the sim_config

sim_config['Integration_and_patterns']['Dither_Patterns']['Dither'] = 'True' # must be str
sim_config['Integration_and_patterns']['Dither_Patterns']['StartInd'] = 1
sim_config['Integration_and_patterns']['Dither_Patterns']['NDither'] = len(ditherXY_arcsec)
sim_config['Integration_and_patterns']['Dither_Patterns']['DitherPat'] = dither_file
sim_config['Pointing_and_Optical_Path']['Pointing_Centre'] = {}
sim_config['Pointing_and_Optical_Path']['Pointing_Centre']['RA'] = telescope_v1_ra
sim_config['Pointing_and_Optical_Path']['Pointing_Centre']['DEC'] = telescope_v1_dec
sim_config['Pointing_and_Optical_Path']['Pointing_Centre']['PosAng'] = -local_roll
                                                                     #<TODO># needs minus here
sim_config


SimConfig([('name', 'Default Simulation'),
           ('Scene', OrderedDict([('filename', 'scene.ini')])),
           ('Observation', OrderedDict([('rel_obsdate', 0.2)])),
           ('Pointing_and_Optical_Path',
            OrderedDict([('pointing_centre',
                          OrderedDict([('ra', 149.84798227082246),
                                       ('dec', 2.3227671659549323),
                                       ('posang', 293.09730273)])),
                         ('Primary_Optical_Path',
                          OrderedDict([('POP', 'IMA'),
                                       ('ConfigPath', 'IMA_FULL')])),
                         ('Pointing_Centre',
                          {'RA': 149.92961,
                           'DEC': 2.43173,
                           'PosAng': -293.0990826383529})])),
           ('Integration_and_patterns',
            OrderedDict([('Dither_Patterns',
                          OrderedDict([('Dither', 'True'),
                          

In [14]:
# prepare output dir and create simulation
# this will create a series of offset and exposure events in `simulation.events`

os.makedirs(sim_output_dir, exist_ok=True)
path_out = sim_output_dir
path_cdp = os.environ['CDP_DIR']

simulation = ObservationSimulation(
    sim_config, scene_config, simulator_config,
    path_out, path_cdp,
)


2022-12-08 18:28:56,677 - INFO - Setting up simulated Observation, with following settings:
2022-12-08 18:28:56,678 - INFO - Configuration Path: IMA_FULL
2022-12-08 18:28:56,679 - INFO - Primary optical path: IMA
2022-12-08 18:28:56,680 - INFO - IMA Filter: F770W
2022-12-08 18:28:56,681 - INFO - IMA Subarray: FULL
2022-12-08 18:28:56,682 - INFO - IMA detector readout mode: FAST
2022-12-08 18:28:56,682 - INFO - IMA detector # exposures: 4
2022-12-08 18:28:56,683 - INFO - IMA detector # integrations: 2
2022-12-08 18:28:56,684 - INFO - IMA detector # frames: 45
2022-12-08 18:28:56,769 - INFO - Simulating dithered observation using pattern: test_mirisim_PA_input/dither.dat
2022-12-08 18:28:56,770 - INFO - Number of dithers defined in dither pattern: 8
2022-12-08 18:28:56,771 - INFO - Starting index of executed dither pattern: 1
2022-12-08 18:28:56,772 - INFO - Number of executed dither positions: 8
2022-12-08 18:28:56,775 - INFO - Reading 'DISTORTION' model from '/Users/dzliu/Data/JWST-MIR

In [15]:
# print info about exposure events

import mirage # pip install --upgrade git+https://github.com/spacetelescope/mirage.git
from mirage.utils import siaf_interface
from mirage.utils.set_telescope_pointing_separated import compute_local_roll

for i in range(len(simulation.events)):
    if isinstance(simulation.events[i], mirisim.obssim.event.ExposureEvent):
        dither_id = simulation.events[i].seq_id
        pointing = simulation.events[i].pointing
        
        # following "obssim/event.py" `update_pointing()`
        v2_off, v3_off = pointing.get_v2v3_offset_actual() # offsets in arcsec in v2v3 coord
        v2_ref, v3_ref = pointing.v2v3_ref # tel v2v3 arcsec coord
        v2_pref, v3_pref = pointing.v2v3_pref # sci pixel coord
        v1_ra, v1_dec = pointing.radec_v1
        pa = pointing.pa
        idlx_off, idly_off = MIRIM_FULL_SIAF.tel_to_idl(-v2_off+v2_ref, -v3_off+v3_ref)
        print('dither seq_id: {}, '
              'idl_off: {:.3f} {:.3f}, '
              'v2v3_off: {:.3f} {:.3f}, '
              'v2v3_ref: {:.3f} {:.3f}, '
              'v2v3_pref: {:.3f} {:.3f}, '
              'pa: {:.3f}'
              .format(dither_id, 
                      idlx_off, idly_off, 
                      v2_off, v3_off, v2_ref, v3_ref, v2_pref, v3_pref, pa)
             )
        
        #v2v3_to_sky = mirisim.wcs.v2v3SkySimTransform(v2_ref, v3_ref, v2_off, v3_off, pa)
        
        # compute local roll using mirage
        local_roll_2 = compute_local_roll(
            pa, 
            v1_ra, v1_dec, 
            v2_ref, v3_ref,
        )
        print('local_roll: {:.3f}, v3_pa: {:.3f}, '
              'v1_ra: {:.7f}, v1_dec: {:.7f}, v2_ref: {:.3f}, v3_ref: {:.3f}'
              .format(
                  local_roll_2, telescope_v3_pa, 
                  v1_ra, v1_dec, v2_ref, v3_ref,
        ))
        print('radec_ref: {}'.format(pointing.radec_ref))
        print('radec_pref: {}'.format(pointing.radec_pref))
        
        print("MIRIM_FULL_SIAF.tel_to_sky(v2_ref, v3_ref)", MIRIM_FULL_SIAF.tel_to_sky(v2_ref, v3_ref))
        
        # update (DO NOT DO UPDATE HERE)
        #simulation.events[i].pointing.pa = local_roll
        #simulation.events[i].pointing.set_radec_v1(telescope_v1_ra, telescope_v1_dec)
        #simulation.events[i].pointing.set_radec_pointing_ref(ra_pref, dec_pref) # keep it MIRI FULL center


dither seq_id: 1, idl_off: 0.000 0.000, v2v3_off: 0.000 0.000, v2v3_ref: -453.559 -373.814, v2v3_pref: -453.559 -373.814, pa: -293.099
local_roll: 66.900, v3_pa: 293.097, v1_ra: -0.0460843, v1_dec: 0.1566256, v2_ref: -453.559, v3_ref: -373.814
radec_ref: (149.92961, 2.43173)
radec_pref: (0.0, 0.0)
MIRIM_FULL_SIAF.tel_to_sky(v2_ref, v3_ref) (149.9296099999603, 2.4317299999969433)
dither seq_id: 2, idl_off: -24.776 -0.974, v2v3_off: -24.606 3.058, v2v3_ref: -453.559 -373.814, v2v3_pref: -453.559 -373.814, pa: -293.099
local_roll: 66.900, v3_pa: 293.097, v1_ra: -0.0495472, v1_dec: 0.1506718, v2_ref: -453.559, v3_ref: -373.814
radec_ref: (149.92961, 2.43173)
radec_pref: (-0.0034628856911664793, -0.005953776968933444)
MIRIM_FULL_SIAF.tel_to_sky(v2_ref, v3_ref) (149.9296099999603, 2.4317299999969433)
dither seq_id: 3, idl_off: -24.459 5.021, v2v3_off: -24.795 -2.941, v2v3_ref: -453.559 -373.814, v2v3_pref: -453.559 -373.814, pa: -293.099
local_roll: 66.900, v3_pa: 293.097, v1_ra: -0.0480349,

In [16]:
# raise NotImplementedError()

In [17]:
# Run simulations 
# 
# In "obssim/obssim.py"
#   simulation.run()
# is just calling each event.simulate()
# 

from mirisim.constants import INS_IMA, DET_IMA
from mirisim.utils import ensure_path_exists
from jwst.assign_wcs.util import calc_rotation_matrix

for event in simulation.events:
    if isinstance(event, mirisim.obssim.event.ExposureEvent):
        # here originally `event.simulate()` is called.
        # but we unfold `event.simulate()` following "obssim/event.py" `event.simulate()`
        start_time = event.clock.get_current_time()
        elapsed_time_ima = datetime.timedelta(seconds=0.0)
        exposure = event.exposures[INS_IMA]
        # Here originally `elapsed_time_ima += exposure.simulate(start_time)` is called,
        # but we unfold `exposure.simulate()` following "obssim/exposure.py" `ImaExposure.simulate()`.
        # Here originally `exposure.create_illummodels_and_detimages(start_time, "Imager")` is called,
        # but we unfold its steps.
        exp_type = "Imager"
        exposure.start_time = start_time
        
        exposure.log.info("Simulating detector illumination for {} exposures for pointing {}"
                          "".format(exp_type, exposure.seq_id))
        ensure_path_exists(os.path.join(exposure.path_out, exposure.path_illum_models))
        # Here originally `exposure.create_illum_models()` is called,
        # but we unfold its steps.
        illum_model = mirisim.imsim.run_imsim(exposure.scene, exposure.subarray, 
                                              exposure.filtername, exposure.pointing,
                                              exposure.rel_obsdate, exposure.simulator_config)
        exposure.update_metadata_illum_model(illum_model, DET_IMA, 
                                             filtername=exposure.filtername, set_imager_wcs=True)
        # Here is the trick! Set correct WCS here
        crval = MIRIM_FULL_SIAF.tel_to_sky(v2_ref, v3_ref)
        crpix = MIRIM_FULL_SIAF.idl_to_sci(0.0, 0.0)
        #pcangle = np.deg2rad(local_roll - MIRIM_FULL_SIAF.VIdlParity * MIRIM_FULL_SIAF.V3IdlYAngle)
        pc = calc_rotation_matrix(
            np.deg2rad(local_roll), 
            np.deg2rad(MIRIM_FULL_SIAF.V3IdlYAngle), 
            MIRIM_FULL_SIAF.VIdlParity)
        pc = np.array(pc).reshape([-1, 2])
        illum_model.set_wcs_metadata(
            crval=crval,
            crpix=crpix,
            ctype=['RA---TAN','DEC--TAN'], 
            cunit=['deg','deg'],
            cdelt=[v2_pixsc/3600., v3_pixsc/3600.],
            pc=pc,
            v3yangle=MIRIM_FULL_SIAF.V3IdlYAngle,
            ra_ref=crval[0],
            dec_ref=crval[1],
            roll_ref=local_roll, # this is from telescope_v3_pa and not accounting for v3yangle
        ) 
        # see https://www.miricle.org/doc/html/_modules/miri/datamodels/miri_model_base.html
        illum_model.set_pointing_metadata(
            ra_v1=telescope_v1_ra, 
            dec_v1=telescope_v1_dec, 
            pa_v3=telescope_v3_pa, # TODO: telescope_v3_pa or local_roll?
        )
        # Continue original code
        fn_part = "{}_{}".format(DET_IMA, exposure.filtername)
        exposure.write_illum_model(illum_model, fn_part)
        exposure.illum_model = illum_model
        
        exposure.log.info("Simulating integrated detector images for {} exposures for pointing {}"
                          "".format(exp_type, exposure.seq_id))
        ensure_path_exists(os.path.join(exposure.path_out, exposure.path_det_images))
        exposure.create_detector_images()
        
        elapsed_time = exposure.elapsed_time
        event.advance_clock(elapsed_time)
    else:
        event.simulate()


2022-12-08 18:31:18,162 - INFO - Simulating detector illumination for Imager exposures for pointing 1
2022-12-08 18:31:18,163 - INFO - Running ImSim.
2022-12-08 18:31:18,164 - INFO - Running MirimImager (version=36) for subarray=FULL
2022-12-08 18:31:18,166 - INFO - Reading 'AREA' model from '/Users/dzliu/Data/JWST-MIRISIM/CDP/MIRI_FM_MIRIMAGE_AREA_07.00.00.fits'
2022-12-08 18:31:18,230 - INFO - Retrieving PCE CDP for filter : F770W
2022-12-08 18:31:18,232 - INFO - Reading 'PCE' model from '/Users/dzliu/Data/JWST-MIRISIM/CDP/MIRI_FM_MIRIMAGE_F770W_PCE_07.00.00.fits'
2022-12-08 18:31:18,323 - INFO - Reading 'PSF' model from '/Users/dzliu/Data/JWST-MIRISIM/CDP/MIRI_FM_MIRIMAGE_F770W_PSF_07.02.00.fits'
2022-12-08 18:31:18,593 - INFO - Reading 'DISTORTION' model from '/Users/dzliu/Data/JWST-MIRISIM/CDP/MIRI_FM_MIRIMAGE_DISTORTION_07.04.01.fits'
2022-12-08 18:31:18,732 - INFO - Processing point sources for IMAGER 
2022-12-08 18:31:56,011 - INFO - No SkyCube
2022-12-08 18:31:56,014 - INFO - 

2022-12-08 18:32:55,785 - INFO - Adding the DARK calibration from /Users/dzliu/Data/JWST-MIRISIM/CDP/MIRI_FM_MIRIMAGE_FAST_DARK_06.01.00.fits
2022-12-08 18:32:56,619 - INFO - Correcting nonlinearity from MIRI_FM_MIRIMAGE_LINEARITY_06.02.00.fits
2022-12-08 18:32:58,911 - INFO - Output subarray undefined or FULL. SUBSTRT=(1,1), SUBSIZE=(1032,1024)
2022-12-08 18:32:58,912 - INFO - WCS keywords defined as CRPIX1=0, CRPIX2=0
2022-12-08 18:33:01,717 - INFO - Exposure time 249.75s (duration 250.81s) 
2022-12-08 18:33:03,527 - INFO - Wrote detector image: test_mirisim_PA_output/det_images/det_image_seq1_MIRIMAGE_F770Wexp2.fits
2022-12-08 18:33:03,528 - INFO - Simulating Imager exposure 3
2022-12-08 18:33:03,548 - INFO - Running SCASim
2022-12-08 18:33:03,550 - INFO - Simulating  detector readout for MIRIMAGE from illumination data model of shape (1, 1024, 1024).
2022-12-08 18:33:03,562 - INFO - Results will be returned in an exposure data model.
2022-12-08 18:33:03,574 - INFO -   Detector read